In [1]:
import pandas as pd
import numpy as np
import glob
from gplearn.genetic import SymbolicRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
mens = []
womens = []
labels = pd.read_csv('./gender_labels.csv')
for s in glob.glob('/neuro/notebooks/all_data_confounds_remove/*.csv'):
    person = int(s.split('/')[-1].split('_')[0])
    data = pd.read_csv(s)
    data = data.rolling(window=10).mean().dropna()
    if labels[labels['person']==person]['gender'].values[0]=='M':
        mens.append(data)
    else:
        womens.append(data)
mens = pd.concat(mens)
womens = pd.concat(womens)  

In [ ]:
# data = pd.read_csv('../notebooks/filter_with_confounds_dataset.csv')
# data = mens
# region = 'x1'
# X = data.drop([region], axis=1).values
# y = data[region].values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)

In [6]:
loses_by_epoch = {}
data = mens
print(data.shape)
for i in range(41, 48):
    region = 'x'+str(i)
    print(region)
    loses_by_epoch[region] = list()
    X = data.drop([region], axis=1).values
    y = data[region].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)

    N, D_in, H, D_out = 64, 1000, 2, 10
    device='cuda:2'
    X_train_torch = torch.FloatTensor(X_train).to(device)
    y_train_torch = torch.FloatTensor(y_train).to(device)

    X_test_torch = torch.FloatTensor(X_test).to(device)
    y_test_torch = torch.FloatTensor(y_test).to(device)

    model = torch.nn.Sequential(
        torch.nn.Linear(47, 100),
        torch.nn.Softplus(),
        torch.nn.Linear(100, 50),
        torch.nn.Softplus(),
        torch.nn.Linear(50, 25),
        torch.nn.Tanh(),
        torch.nn.Linear(25, 1),
    ).to(device)
    loss_fn = torch.nn.MSELoss(reduction='mean')

    learning_rate = 1e-2
    batch_size = 8096
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for t in range(10000):
        for batch in range(0, X_train.shape[0], batch_size):

            y_pred_train = model(X_train_torch[batch:batch+batch_size])


            loss_train = loss_fn(y_pred_train, y_train_torch.reshape(-1, 1)[batch:batch+batch_size])

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            
        train_predict = model(X_train_torch).detach().cpu().numpy()
        test_predict = model(X_test_torch).detach().cpu().numpy()
        r2_train = r2_score(np.ravel(train_predict), np.ravel(y_train))
        r2_test = r2_score(np.ravel(test_predict), np.ravel(y_test))
        mse_train = mean_squared_error(np.ravel(train_predict), np.ravel(y_train))
        mse_test = mean_squared_error(np.ravel(test_predict), np.ravel(y_test))
        loses_by_epoch[region].append([r2_train, r2_test, mse_train, mse_test])
        if t%2000==0:
            print('Iteration:',t)
            print('train:',r2_score(np.ravel(train_predict), np.ravel(y_train)))
            print('test:',r2_score(np.ravel(test_predict), np.ravel(y_test)))
            print('train:',mean_squared_error(np.ravel(train_predict), np.ravel(y_train)))
            print('test:',mean_squared_error(np.ravel(test_predict), np.ravel(y_test)))
    torch.save(model.state_dict(), './models/pytorch_model_{0}_mens.pt'.format(region))

(147684, 48)
x41
Iteration: 0
train: -4.368909272123169
test: -4.49113944695885
train: 0.27695844880522025
test: 0.28170401002928924
Iteration: 2000
train: 0.9196575221704893
test: 0.8569303294267646
train: 0.0334324030762009
test: 0.05973448594099765
Iteration: 4000
train: 0.9201842444042457
test: 0.8484023689157463
train: 0.032849639256478186
test: 0.06279730958215812
Iteration: 6000
train: 0.9167089256208378
test: 0.8423334585385535
train: 0.03412402047278984
test: 0.06491872583541515
Iteration: 8000
train: 0.9196468609571904
test: 0.8443493100658603
train: 0.033350282218913785
test: 0.06481518514991932
x42
Iteration: 0
train: -19.27966888809776
test: -19.629709694039565
train: 0.32407144347497846
test: 0.32856965092242313
Iteration: 2000
train: 0.8755277769120948
test: 0.7869351932050086
train: 0.046188556544344034
test: 0.07947832585777491
Iteration: 4000
train: 0.8838468050793087
test: 0.7808903833078038
train: 0.04246045068019806
test: 0.08080550705802293
Iteration: 6000
train: 